# **네이버 영화 리뷰 - 정수 인코딩**

구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


토큰화 데이터 불러오기

In [ ]:
import numpy as np

x_train_tokenized = np.load("/content/drive/MyDrive/ml_data/NaverMovieReview/x_train_tokenized.npy", allow_pickle=True)
x_test_tokenized = np.load("/content/drive/MyDrive/ml_data/NaverMovieReview/x_test_tokenized.npy", allow_pickle=True)

In [ ]:
x_train_tokenized[:5]

array([list(['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리']),
       list(['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '않', '구나']),
       list(['너무', '재', '밓었다그래서보는것을추천한다']),
       list(['교도소', '이야기', '구먼', '솔직히', '재미', '없', '평점', '조정']),
       list(['사이몬페그', '익살', '스런', '연기', '돋보였', '던', '영화', '스파이더맨', '에서', '늙', '어', '보이', '기', '만', '했', '던', '커스틴', '던스트', '너무나', '이뻐', '보였'])],
      dtype=object)

In [ ]:
x_test_tokenized[:5]

array([list(['굳', 'ㅋ']),
       list(['뭐', '야', '평점', '나쁘', '진', '않', '지만', '점', '짜리', '더더욱', '아니', '잖아']),
       list(['지루', '않', '은데', '완전', '막장', '돈', '주', '보', '기']),
       list(['만', '아니', '었', '어도', '별', '다섯', '개', '줬', '텐데', '왜', '로', '나와서', '제', '심기', '불편', '죠']),
       list(['음악', '주', '된', '최고', '음악', '영화'])], dtype=object)

단어 집합 분석

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train_tokenized)

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수

In [ ]:
total_cnt

49942

In [ ]:
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if (value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print("단어 집합(vocabulary)의 크기: {:,}".format(total_cnt))
print("등장 빈도가 {}번 이하인 희귀 단어의 수: {:,}개".format(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율: {:.2f}%".format(rare_cnt / total_cnt * 100))
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {:.2f}%".format(rare_freq / total_freq * 100))

단어 집합(vocabulary)의 크기: 49,942
등장 빈도가 1번 이하인 희귀 단어의 수: 21,313개
단어 집합에서 희귀 단어의 비율: 42.68%
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.19%


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train_tokenized)

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if (value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print("단어 집합(vocabulary)의 크기: {:,}".format(total_cnt))
print("등장 빈도가 {}번 이하인 희귀 단어의 수: {:,}개".format(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율: {:.2f}%".format(rare_cnt / total_cnt * 100))
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {:.2f}%".format(rare_freq / total_freq * 100))

단어 집합(vocabulary)의 크기: 49,942
등장 빈도가 1번 이하인 희귀 단어의 수: 21,313개
단어 집합에서 희귀 단어의 비율: 42.68%
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.19%


단어 집합의 최대 크기 제한

In [ ]:
# 희귀 단어(등장 빈도수가 1이하인 단어) 수를 제외
vocab_size = total_cnt - rare_cnt + 2
print("단어 집합의 크기 : {:,}".format(vocab_size))

단어 집합의 크기 : 28,631


정수 인코딩

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = "OOV")
tokenizer.fit_on_texts(x_train_tokenized)
x_train_encoded = tokenizer.texts_to_sequences(x_train_tokenized)
x_test_encoded = tokenizer.texts_to_sequences(x_test_tokenized)

In [ ]:
x_train_encoded[:5]

array([list([20, 58, 888, 26, 206, 7, 21, 692]),
       list([970, 474, 484, 629, 2, 102, 1547, 39, 859, 942, 30, 356]),
       list([11, 184, 1]), list([8029, 134, 4127, 269, 77, 5, 44, 3319]),
       list([21623, 8481, 1044, 39, 2695, 46, 2, 2618, 27, 1111, 22, 318, 29, 9, 28, 46, 15305, 21624, 385, 2843, 1670])],
      dtype=object)

In [ ]:
x_test_encoded[:5]

array([list([781, 119]),
       list([69, 100, 44, 891, 306, 30, 24, 31, 590, 3394, 55, 848]),
       list([80, 30, 209, 116, 320, 129, 41, 3, 29]),
       list([9, 55, 8, 422, 224, 2004, 110, 511, 538, 49, 16, 585, 317, 19685, 785, 268]),
       list([215, 41, 137, 42, 215, 2])], dtype=object)

정수 인코딩 데이터 저장

In [ ]:
x_train_encoded = np.array(x_train_encoded, dtype="object")
x_test_encoded = np.array(x_test_encoded, dtype="object")
np.save("/content/drive/MyDrive/ml_data/NaverMovieReview/x_train_encoded.npy", x_train_encoded, allow_pickle=True)
np.save("/content/drive/MyDrive/ml_data/NaverMovieReview/x_test_encoded.npy", x_test_encoded, allow_pickle=True)